In [22]:
def parse_file(datafile):
    index = 0
    data = []
    header = []
    with open(datafile, "rb") as f:
        for line in f:
            if index == 11:
                break
            line_data = line.split(',')
            line_dict = {}
            if index == 0:
                header = [field.strip() for field in line_data]
            else:
                for i, value in enumerate(line_data):
                    line_dict[header[i]] = value.strip()
                data.append(line_dict)
            index += 1
    return data

In [23]:
data = parse_file('beatles-diskography.csv')
print data[0]
print data[9]
print len(data)

{'Title': 'Please Please Me', 'UK Chart Position': '1', 'Label': 'Parlophone(UK)', 'Released': '22-Mar-63', 'US Chart Position': '-', 'RIAA Certification': 'Platinum', 'BPI Certification': 'Gold'}
{'Title': '', 'UK Chart Position': '1', 'Label': 'Parlophone(UK)', 'Released': '10-Jul-64', 'US Chart Position': '-', 'RIAA Certification': '', 'BPI Certification': 'Gold'}
10


In [28]:
import csv
import pprint

In [25]:
def parse_csv(datafile):
    data = []
    n = 0
    with open(datafile, 'rb') as sd:
        r = csv.DictReader(sd)
        for line in r:
            data.append(line)
    return data

In [27]:
songs = parse_csv('beatles-diskography.csv')

In [29]:
import xlrd

In [31]:
def parse_xls(datafile):
    workbook = xlrd.open_workbook(datafile)
    sheet = workbook.sheet_by_index(0)

    data = [[sheet.cell_value(r, col) 
                for col in range(sheet.ncols)] 
                    for r in range(sheet.nrows)]

    print "\nList Comprehension"
    print "data[3][2]:",
    print data[3][2]

    print "\nCells in a nested loop:"    
    for row in range(sheet.nrows):
        for col in range(sheet.ncols):
            if row == 50:
                print sheet.cell_value(row, col),


    ### other useful methods:
    print "\nROWS, COLUMNS, and CELLS:"
    print "Number of rows in the sheet:", 
    print sheet.nrows
    print "Type of data in cell (row 3, col 2):", 
    print sheet.cell_type(3, 2)
    print "Value in cell (row 3, col 2):", 
    print sheet.cell_value(3, 2)
    print "Get a slice of values in column 3, from rows 1-3:"
    print sheet.col_values(3, start_rowx=1, end_rowx=4) #not including row 4

    print "\nDATES:"
    print "Type of data in cell (row 1, col 0):", 
    print sheet.cell_type(1, 0)
    exceltime = sheet.cell_value(1, 0)
    print "Time in Excel format:",
    print exceltime
    print "Convert time to a Python datetime tuple, from the Excel float:",
    print xlrd.xldate_as_tuple(exceltime, 0)

    return data

In [32]:
ercot = parse_xls('2013_ERCOT_Hourly_Load_Data.xls')


List Comprehension
data[3][2]: 1036.088697

Cells in a nested loop:
41277.0833333 9238.73731 1438.20528 1565.442856 916.708348 14010.903488 3027.98334 6165.211119 1157.741663 37520.933404 
ROWS, COLUMNS, and CELLS:
Number of rows in the sheet: 7296
Type of data in cell (row 3, col 2): 2
Value in cell (row 3, col 2): 1036.088697
Get a slice of values in column 3, from rows 1-3:
[1411.7505669999982, 1403.4722870000019, 1395.053150000001]

DATES:
Type of data in cell (row 1, col 0): 3
Time in Excel format: 41275.0416667
Convert time to a Python datetime tuple, from the Excel float: (2013, 1, 1, 1, 0, 0)


In [35]:
"""
Your task is as follows:
- read the provided Excel file
- find and return the min, max and average values for the COAST region
- find and return the time value for the min and max entries
- the time values should be returned as Python tuples

Please see the test function for the expected return format
"""

'\nYour task is as follows:\n- read the provided Excel file\n- find and return the min, max and average values for the COAST region\n- find and return the time value for the min and max entries\n- the time values should be returned as Python tuples\n\nPlease see the test function for the expected return format\n'

In [49]:
def parse_ercot(datafile):
    workbook = xlrd.open_workbook(datafile)
    sheet = workbook.sheet_by_index(0)
    data = {
            'maxtime': (0, 0, 0, 0, 0, 0),
            'maxvalue': 0,
            'mintime': (0, 0, 0, 0, 0, 0),
            'minvalue': 0,
            'avgcoast': 0
    }
    coast_data = [float(sheet.cell_value(r, 1)) for r in range(1, sheet.nrows)] #coast data is in 2nd row
    data['maxvalue'] = max(coast_data)
    data['minvalue'] = min(coast_data)
    data['avgcoast'] = sum(coast_data)/float(len(coast_data))
    max_row = coast_data.index(max(coast_data)) + 1 #add one as first one in excel sheet is header row
    min_row = coast_data.index(min(coast_data)) + 1
    data['maxtime'] = xlrd.xldate_as_tuple(sheet.cell_value(max_row, 0), 0)
    data['mintime'] = xlrd.xldate_as_tuple(sheet.cell_value(min_row, 0), 0)
    return data

In [52]:
coast = parse_ercot('2013_ERCOT_Hourly_Load_Data.xls')
pprint.pprint(coast)

{'avgcoast': 10976.933460679751,
 'maxtime': (2013, 8, 13, 17, 0, 0),
 'maxvalue': 18779.025510000003,
 'mintime': (2013, 2, 3, 4, 0, 0),
 'minvalue': 6602.113898999982}


In [53]:
import requests
import json

In [54]:
BASE_URL = "http://musicbrainz.org/ws/2/"
ARTIST_URL = BASE_URL + "artist/"

query_type = {  "simple": {},
                "atr": {"inc": "aliases+tags+ratings"},
                "aliases": {"inc": "aliases"},
                "releases": {"inc": "releases"}}

In [55]:
def query_site(url, params, uid="", fmt="json"):
    params["fmt"] = fmt
    r = requests.get(url + uid, params=params)
    print "requesting", r.url

    if r.status_code == requests.codes.ok:
        return r.json()
    else:
        r.raise_for_status()


def query_by_name(url, params, name):
    params["query"] = "artist:" + name
    return query_site(url, params)

In [56]:
def pretty_print(data, indent=4):
    if type(data) == dict:
        print json.dumps(data, indent=indent, sort_keys=True)
    else:
        print data

In [57]:
def brainz(name):
    results = query_by_name(ARTIST_URL, query_type["simple"], name)
    pretty_print(results)

    artist_id = results["artists"][1]["id"]
    print "\nARTIST:"
    pretty_print(results["artists"][1])

    artist_data = query_site(ARTIST_URL, query_type["releases"], artist_id)
    releases = artist_data["releases"]
    print "\nONE RELEASE:"
    pretty_print(releases[0], indent=2)
    release_titles = [r["title"] for r in releases]

    print "\nALL TITLES:"
    for t in release_titles:
        print t

In [69]:
def query(name):
    results = query_by_name(ARTIST_URL, query_type["simple"], name)
    #artist_id = results["artists"][1]["id"]
    return results

In [81]:
first = query("First Aid Kit")["artists"]
names = [band['name'] for band in first]
fak = [name for name in names if name=="First Aid Kit"]
fak

requesting http://musicbrainz.org/ws/2/artist/?query=artist%3AFirst+Aid+Kit&fmt=json


[u'First Aid Kit', u'First Aid Kit']

In [70]:
queen = query("Queen")
pretty_print(queen)

requesting http://musicbrainz.org/ws/2/artist/?query=artist%3AQueen&fmt=json
{
    "artists": [
        {
            "aliases": [
                {
                    "begin-date": "2011", 
                    "end-date": null, 
                    "locale": null, 
                    "name": "Queen + Adam Lambert", 
                    "primary": null, 
                    "sort-name": "Queen + Adam Lambert", 
                    "type": null
                }
            ], 
            "area": {
                "id": "8a754a16-0027-3a29-b6d7-2b40ea0481ed", 
                "name": "United Kingdom", 
                "sort-name": "United Kingdom"
            }, 
            "begin-area": {
                "id": "f03d09b3-39dc-4083-afd6-159e3f0d462f", 
                "name": "London", 
                "sort-name": "London"
            }, 
            "country": "GB", 
            "disambiguation": "UK rock group", 
            "id": "0383dadf-2a4e-4d10-a46a-e9e041da8eb3", 
         

In [78]:
od = query("One Direction")
od['artists'][0]['disambiguation']

requesting http://musicbrainz.org/ws/2/artist/?query=artist%3AOne+Direction&fmt=json


u'English-Irish boy band formed in 2010'

In [82]:
beatles = query("Beatles")

requesting http://musicbrainz.org/ws/2/artist/?query=artist%3ABeatles&fmt=json


In [83]:
pretty_print(beatles)

{
    "artists": [
        {
            "aliases": [
                {
                    "begin-date": null, 
                    "end-date": null, 
                    "locale": "ko", 
                    "name": "\ub354 \ube44\ud2c0\uc988", 
                    "primary": true, 
                    "sort-name": "\ub354 \ube44\ud2c0\uc988", 
                    "type": null
                }, 
                {
                    "begin-date": null, 
                    "end-date": null, 
                    "locale": "ja", 
                    "name": "\u30b6\u30fb\u30d3\u30fc\u30c8\u30eb\u30ba", 
                    "primary": true, 
                    "sort-name": "\u30d3\u30fc\u30c8\u30eb\u30ba (\u30b6)", 
                    "type": null
                }, 
                {
                    "begin-date": null, 
                    "end-date": null, 
                    "locale": "en", 
                    "name": "Beatles", 
                    "primary": null, 
        

In [ ]:
"""
Your task is to process the supplied file and use the csv module to extract data from it.
The data comes from NREL (National Renewable Energy Laboratory) website. Each file
contains information from one meteorological station, in particular - about amount of
solar and wind energy for each hour of day.

Note that the first line of the datafile is neither data entry, nor header. It is a line
describing the data source. You should extract the name of the station from it.

The data should be returned as a list of lists (not dictionaries).
You can use the csv modules "reader" method to get data in such format.
Another useful method is next() - to get the next line from the iterator.
You should only change the parse_file function.
"""

In [91]:
def parse_nrel(datafile):
    data = []
    name = ""
    with open(datafile, 'rb') as f:
        r = csv.reader(f)
        for line in r:
            if r.line_num == 1:
                name = line[1]
            elif r.line_num > 2:
                data.append(line)
    return (name, data)

In [92]:
station, data = parse_nrel('745090.csv')

In [95]:
print station
print data[0][1]
print data[2][0] 
print data[2][5] 

MOUNTAIN VIEW MOFFETT FLD NAS
01:00
01/01/2005
2


In [ ]:
# Find the time and value of max load for each of the regions
# COAST, EAST, FAR_WEST, NORTH, NORTH_C, SOUTHERN, SOUTH_C, WEST
# and write the result out in a csv file, using pipe character | as the delimiter.
# An example output can be seen in the "example.csv" file.

In [120]:
def parse_file(datafile):
    workbook = xlrd.open_workbook(datafile)
    sheet = workbook.sheet_by_index(0)
    regions = ['COAST', 'EAST', 'FAR_WEST', 'NORTH', 'NORTH_C', 'SOUTHERN', 'SOUTH_C', 'WEST']
    data = {}
    # Remember that you can use xlrd.xldate_as_tuple(sometime, 0) to convert
    # Excel date to Python tuple of (year, month, day, hour, minute, second)
    for i, region in enumerate(regions):
        rdict = {}
        rdata = [float(sheet.cell_value(r, i+1)) for r in range(1, sheet.nrows)] #data for region
        max_load = max(rdata)
        max_row = rdata.index(max_load) +1
        maxday = xlrd.xldate_as_tuple(sheet.cell_value(max_row, 0), 0) #convert excel date to Python tuple
        rdict = {'Max Load': max_load,
                       'Year': maxday[0],
                        'Month': maxday[1],
                        'Day': maxday[2],
                        'Hour': maxday[3]}
        data[region] = rdict
    return data

In [122]:
maxregion = parse_file('2013_ERCOT_Hourly_Load_Data.xls')
maxregion['FAR_WEST']

{'Day': 26,
 'Hour': 17,
 'Max Load': 2281.2722140000024,
 'Month': 6,
 'Year': 2013}

In [123]:
def save_file(data, filename):
    with open(filename, 'wb') as csvfile:
        fieldnames = ['Station', 'Year', 'Month', 'Day', 'Hour', 'Max Load']
        writer = csv.DictWriter(csvfile, delimiter='|', fieldnames=fieldnames)
        writer.writeheader()
        for s in data.keys():
            year, month, day, hour, maxload = data[s]['Year'], data[s]['Month'], data[s]['Day'], data[s]['Hour'], data[s]['Max Load']
            writer.writerow({'Station':s, 'Year': year, 'Month': month, 'Day': day, 'Hour': hour, 'Max Load': maxload})

In [124]:
save_file(maxregion, 'maxload.csv')

In [125]:
import codecs

In [127]:
def get_from_file(filename):
    with open(filename, "r") as f:
        return json.loads(f.read())

In [146]:
def article_overview(filename):
    data = get_from_file(filename)
    titles = []
    urls =[]
    for article in data:
        titles.append({article['section']: article['title']}) 
        pics = article['media'] #list of all media in a particular article
        for p in pics:
            metadata = p['media-metadata']
            for m in metadata:
                if m['format'] == 'Standard Thumbnail':
                    urls.append(m['url'])
    return (titles, urls)

In [129]:
nyt = get_from_file('nytimes.txt')

In [132]:
print 'Number of articles in file:', len(nyt)

Number of articles in file: 20


In [136]:
nyt[0]

{u'abstract': u'Couples can be happier now than ever before. But it\u2019s rare.',
 u'adx_keywords': u'Marriages;Divorce, Separations and Annulments;Social Conditions and Trends;Sociology',
 u'asset_id': 100000002711650.0,
 u'byline': u'By ELI J. FINKEL',
 u'column': u'Opinion',
 u'des_facet': [u'MARRIAGES',
  u'DIVORCE, SEPARATIONS AND ANNULMENTS',
  u'SOCIAL CONDITIONS AND TRENDS',
  u'SOCIOLOGY'],
 u'geo_facet': u'',
 u'id': u'100000002711649',
 u'media': [{u'caption': u'',
   u'copyright': u'Pieter Van Eenoge',
   u'media-metadata': [{u'format': u'Standard Thumbnail',
     u'height': 75,
     u'url': u'http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16COVER/16COVER-thumbStandard.jpg',
     u'width': 75},
    {u'format': u'superJumbo',
     u'height': 1552,
     u'url': u'http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16COVER/16COVER-superJumbo.jpg',
     u'width': 2048},
    {u'format': u'Large',
     u'height': 263,
     u'url': u'http://graphics8.nytime

In [147]:
titles, urls = article_overview('nytimes.txt')

In [148]:
titles

[{u'Opinion': u'The All-or-Nothing Marriage'},
 {u'Opinion': u'Marry First, Then Cheat'},
 {u'Opinion': u'Professors, We Need You!'},
 {u'N.Y. / Region': u'Common Core Curriculum Now Has Critics on the Left'},
 {u'Fashion & Style': u'For Interns, All Work and No Payoff'},
 {u'Sports': u'NBC Pushes Too Far in Bringing Bode Miller to Tears'},
 {u'Opinion': u'Barons of Broadband'},
 {u'Sports': u'A Mad Dash for Salt Rescues Olympic Slopes'},
 {u'U.S.': u'Spying by N.S.A. Ally Entangled U.S. Law Firm'},
 {u'N.Y. / Region': u'Addicted on Staten Island'},
 {u'Business Day': u'Saving an Endangered British Species: The Pub'},
 {u'Sports': u'Behind Meryl Davis and Charlie White, U.S. Is Close to Its First Ice Dance Gold'},
 {u'Technology': u'Intel\u2019s Sharp-Eyed Social Scientist'},
 {u'Sports': u'Hanyu Falls Twice, but Still Wins Gold'},
 {u'Opinion': u'Is the Universe a Simulation?'},
 {u'Opinion': u'Parental Pity Party'},
 {u'Opinion': u'Behind Bars'},
 {u'Opinion': u'A Watchful Eye in Hos

In [149]:
urls

[u'http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16COVER/16COVER-thumbStandard.jpg',
 u'http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16MARRIAGEJP/16MARRIAGEJP-thumbStandard.jpg',
 u'http://graphics8.nytimes.com/images/2014/02/16/sunday-review/16DOWD/16DOWD-thumbStandard.jpg',
 u'http://graphics8.nytimes.com/images/2010/09/16/opinion/Dowd_New/Dowd_New-thumbStandard.jpg',
 u'http://graphics8.nytimes.com/images/2010/09/16/opinion/Kristof_New/Kristof_New-thumbStandard-v4.jpg',
 u'http://graphics8.nytimes.com/images/2014/02/17/nyregion/commoncore/commoncore-thumbStandard-v2.jpg',
 u'http://graphics8.nytimes.com/images/2014/02/17/nyregion/JP-COMMONCORE1/JP-COMMONCORE1-thumbStandard.jpg',
 u'http://graphics8.nytimes.com/images/2014/02/17/nyregion/JP-COMMONCORE2/JP-COMMONCORE2-thumbStandard.jpg',
 u'http://graphics8.nytimes.com/images/2014/02/16/fashion/16INTERNS_SPAN/16INTERNS_SPAN-thumbStandard.jpg',
 u'http://graphics8.nytimes.com/images/2014/02/16/fashion/16J